In [20]:
import pandas as pd,os
import numpy as np

df1 = pd.read_json(os.getcwd()+'/datasets/subtaskA_train_monolingual.jsonl', lines=True)
df1 = df1[['text', 'label']]

print(df1.shape)

(119757, 2)


In [21]:
df2 = pd.read_json(os.getcwd()+'/datasets/subtaskA_dev_monolingual.jsonl', lines=True)
df2 = df2[['text', 'label']]

print(df2.shape)

(5000, 2)


In [22]:
combined_df = pd.concat([df1, df2], ignore_index=True)

print(combined_df.head())
print(combined_df.tail())
print(combined_df.shape)

print(df1.shape[0]+df2.shape[0])

                                                text  label
0  Forza Motorsport is a popular racing game that...      1
1  Buying Virtual Console games for your Nintendo...      1
2  Windows NT 4.0 was a popular operating system ...      1
3  How to Make Perfume\n\nPerfume is a great way ...      1
4  How to Convert Song Lyrics to a Song'\n\nConve...      1
                                                     text  label
124752  The paper deals with an interesting applicatio...      0
124753  This manuscript tries to tackle neural network...      0
124754  The paper introduced a regularization scheme t...      0
124755  Inspired by the analysis on the effect of the ...      0
124756  \n- You definitely need to report misclassific...      0
(124757, 2)
124757


In [23]:
'''loading glove'''
embeddings_index={}
with open('../0 playground and indoor/OtherData/glove.6B.200d.txt','r',encoding='utf-8') as f:
    for line in f:
        values=line.split()
        word=values[0]
        vectors=np.asarray(values[1:],'float32')
        embeddings_index[word]=vectors
f.close()
print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [24]:
'''glove building'''

from nltk.tokenize import word_tokenize
from tqdm import tqdm 

def sent2vec(s):
    """ Function Creates a normalized vector for the whole sentence"""
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(200)
    return v / np.sqrt((v ** 2).sum())

print('Complete df:')
df_x = np.array([sent2vec(x) for x in tqdm(combined_df['text'])])
print(df_x.shape)
df_y=combined_df['label']


Complete df:


100%|██████████| 124757/124757 [03:04<00:00, 674.55it/s] 

(124757, 200)


In [25]:
from sklearn.preprocessing import StandardScaler
# Initialize the StandardScaler
scaler = StandardScaler()
# Fit the scaler to the training data and transform the data
train_x = scaler.fit_transform(df_x)

In [27]:
import pickle

# Save the trained scaler for later use if needed
with open('datasets/subtaskA_scaler_glove_train_dev_monolingual.pkl', 'wb') as scaler_file:
    pickle.dump(scaler, scaler_file)

In [28]:
print(type(df_x))
print(type(df_y))
print(df_x.shape)
print(df_y.shape)

<class 'numpy.ndarray'>
<class 'pandas.core.series.Series'>
(124757, 200)
(124757,)


In [29]:
'''Split back in train and dev'''

# Splitting into training and development sets
train_x = df_x[:df1.shape[0]]
train_y = df_y[:df1.shape[0]]
dev_x = df_x[df1.shape[0]:df1.shape[0]+df2.shape[0]]
dev_y = df_y[df1.shape[0]:df1.shape[0]+df2.shape[0]]

# Printing the shapes to confirm the splits
print(train_x.shape)
print(train_y.shape)
print(dev_x.shape)
print(dev_y.shape)

(119757, 200)
(119757,)
(5000, 200)
(5000,)


In [31]:
'''Save it for later'''

import pickle
# Creating a dictionary to hold all datasets
datasets = {
    'train_x': train_x,
    'train_y': train_y,
    'dev_x': dev_x,
    'dev_y': dev_y
}

# Saving the datasets into a pickle file
with open('datasets/subtaskA_glove_train_dev_monolingual.pkl', 'wb') as f:
    pickle.dump(datasets, f)